In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64, os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from animal_shelter import AnimalShelter

In [2]:
USERNAME = "aacuser"
PASSWORD = "snhu1234"
HOST     = "nv-desktop-services.apporto.com"
PORT     = 33346
DB_NAME  = "AAC"
COLL     = "animals"

db = AnimalShelter(USERNAME, PASSWORD, HOST, PORT, DB_NAME, COLL)
df = pd.DataFrame.from_records(db.read({}, projection=None))
if "_id" in df.columns:
    df = df.drop(columns=["_id"])

def parse_age_to_weeks(age_text):
    if not isinstance(age_text, str) or not age_text.strip():
        return None
    text = age_text.strip().lower()
    m = re.match(r"(\d+)\s+(year|years|month|months|week|weeks|day|days)", text)
    if not m:
        return None
    qty = int(m.group(1))
    unit = m.group(2)
    if "year" in unit:
        return qty * 52
    if "month" in unit:
        return qty * 4.348
    if "week" in unit:
        return qty
    if "day" in unit:
        return qty / 7.0
    return None


In [3]:
RESCUE_SPECS = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": ["Intact Female"],
        "age_weeks": (26, 156)
    },
    "Mountain or Wilderness Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": ["Intact Male"],
        "age_weeks": (26, 156)
    },
    "Disaster or Individual Tracking": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": ["Intact Male"],
        "age_weeks": (20, 300)
    }
}

def build_mongo_query(rescue_type):
    if rescue_type is None or rescue_type == "Reset":
        return {"animal_type": "Dog"}
    spec = RESCUE_SPECS.get(rescue_type)
    if not spec:
        return {"animal_type": "Dog"}
    breeds = spec["breeds"]
    sexes  = spec["sex"]
    lo, hi = spec["age_weeks"]
    query = {
        "$and": [
            {"animal_type": "Dog"},
            {"breed": {"$in": breeds}},
            {"sex_upon_outcome": {"$in": sexes}},
            {"age_upon_outcome_in_weeks": {"$gte": lo, "$lte": hi}}
        ]
    }
    return query


In [4]:
app = JupyterDash("ProjectTwoDashboard")

logo_path = os.path.join("assets", "gs_logo.png")
inline_logo_src = None
if os.path.exists(logo_path):
    logo_src = "/assets/gs_logo.png"
else:
    try:
        with open("gs_logo.png", "rb") as f:
            inline_logo_src = "data:image/png;base64," + base64.b64encode(f.read()).decode()
    except:
        inline_logo_src = None

controls = html.Div([
    html.Div([
        html.Img(src=(logo_src if 'logo_src' in locals() else inline_logo_src),
                 style={"height": "60px", "marginRight": "12px"}) if (('logo_src' in locals()) or inline_logo_src) else html.Span(),
        html.H1("Grazioso Salvare: Search & Rescue Candidate Dashboard",
                style={"display": "inline-block", "verticalAlign": "middle", "margin": "0"})
    ], style={"display": "flex", "alignItems": "center", "gap": "12px"}),
    html.Div("Built by: Connor Holohan — CS-340 Project Two", style={"marginTop": "6px", "fontStyle": "italic"}),
    html.Hr(),
    html.Div([
        html.Label("Rescue Type Filter:", style={"fontWeight": "bold"}),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Reset (All Dogs)", "value": "Reset"},
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
            ],
            value="Reset",
            labelStyle={"display": "block"}
        )
    ], style={"width": "280px"}),
    html.Hr()
])

data_table = dash_table.DataTable(
    id='datatable-id',
    columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df.columns] if len(df) else [],
    data=df.to_dict('records') if len(df) else [],
    editable=False,
    filter_action="native",
    sort_action="native",
    sort_mode="multi",
    column_selectable="single",
    row_selectable="single",
    row_deletable=False,
    selected_columns=[],
    selected_rows=[],
    page_action="native",
    page_current=0,
    page_size=12,
    style_table={"overflowX": "auto"},
    style_cell={"textAlign": "left", "minWidth": "120px", "width": "120px", "maxWidth": "260px"},
    style_data={"whiteSpace": "normal", "height": "auto"}
)

graphs_row = html.Div(className='row', style={'display': 'flex', 'gap': '12px'}, children=[
    html.Div(id='graph-id', className='col s12 m6', style={"flex": "1 1 50%"}),
    html.Div(id='map-id',   className='col s12 m6', style={"flex": "1 1 50%"})
])

app.layout = html.Div([controls, data_table, html.Br(), html.Hr(), graphs_row])


In [5]:
@app.callback(Output('datatable-id', 'data'),
              Output('datatable-id', 'columns'),
              Input('filter-type', 'value'))
def update_table(filter_type):
    query = build_mongo_query(filter_type)
    records = db.read(query) if query else []
    if filter_type != "Reset" and not records:
        spec = RESCUE_SPECS.get(filter_type, {})
        breeds = spec.get("breeds", [])
        sexes  = spec.get("sex", [])
        lo, hi = spec.get("age_weeks", (0, 10000))
        fallback_q = {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$in": breeds}},
                {"sex_upon_outcome": {"$in": sexes}}
            ]
        }
        records = db.read(fallback_q)
        df_local = pd.DataFrame.from_records(records)
        if not df_local.empty:
            if "_id" in df_local.columns:
                df_local = df_local.drop(columns=["_id"])
            if "age_upon_outcome_in_weeks" not in df_local.columns:
                df_local["age_upon_outcome_in_weeks"] = df_local["age_upon_outcome"].apply(parse_age_to_weeks)
            df_local = df_local[
                (df_local["age_upon_outcome_in_weeks"].fillna(-1) >= lo) &
                (df_local["age_upon_outcome_in_weeks"].fillna(-1) <= hi)
            ]
            data = df_local.to_dict('records')
            cols = [{"name": c, "id": c, "deletable": False, "selectable": True} for c in df_local.columns]
            return data, cols
        else:
            return [], []
    df_out = pd.DataFrame.from_records(records)
    if not df_out.empty and "_id" in df_out.columns:
        df_out = df_out.drop(columns=["_id"])
    data = df_out.to_dict('records') if not df_out.empty else []
    cols = [{"name": c, "id": c, "deletable": False, "selectable": True} for c in (df_out.columns if not df_out.empty else [])]
    return data, cols

@app.callback(Output('graph-id', "children"),
              Input('datatable-id', "derived_virtual_data"))
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData) if viewData else pd.DataFrame()
    if dff.empty or "breed" not in dff.columns:
        return [html.Div("No data to plot.", style={"padding": "12px"})]
    top = dff["breed"].fillna("Unknown").value_counts().reset_index().rename(columns={"index": "breed", "breed": "count"}).head(15)
    fig = px.bar(top, x="breed", y="count", title="Breed distribution of current results")
    fig.update_layout(xaxis_title="Breed", yaxis_title="Count", xaxis={"tickangle": -45})
    return [dcc.Graph(figure=fig)]

@app.callback(Output('datatable-id', 'style_data_conditional'),
              Input('datatable-id', 'selected_columns'))
def highlight_columns(selected_columns):
    selected_columns = selected_columns or []
    return [{'if': {'column_id': col}, 'background_color': '#D2F3FF'} for col in selected_columns]

@app.callback(Output('map-id', "children"),
              Input('datatable-id', "derived_virtual_data"),
              Input('datatable-id', "derived_virtual_selected_rows"))
def update_map(viewData, index):
    if not viewData:
        return [html.Div("No data to map.", style={"padding": "12px"})]
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.Div("No data to map.", style={"padding": "12px"})]
    row_idx = index[0] if index else 0
    row_idx = min(max(row_idx, 0), len(dff) - 1)
    lat_col = next((c for c in dff.columns if c.lower() in ("location_lat", "lat", "latitude")), None)
    lon_col = next((c for c in dff.columns if c.lower() in ("location_long", "location_lng", "lon", "lng", "longitude")), None)
    breed_col = "breed" if "breed" in dff.columns else None
    name_col  = "name"  if "name"  in dff.columns else None
    center_lat, center_lon = 30.75, -97.48
    lat = dff.iloc[row_idx][lat_col] if lat_col in dff.columns else None
    lon = dff.iloc[row_idx][lon_col] if lon_col in dff.columns else None
    breed = dff.iloc[row_idx][breed_col] if breed_col else "Breed"
    name  = dff.iloc[row_idx][name_col]  if name_col  else "Unknown"
    try:
        lat = float(lat)
        lon = float(lon)
    except (TypeError, ValueError):
        lat, lon = center_lat, center_lon
    return [dl.Map(style={'width': '100%', 'height': '500px'},
                   center=[lat if lat else center_lat, lon if lon else center_lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(str(breed)),
            dl.Popup([html.H4("Animal Name"), html.P(str(name))])
        ])
    ])]


In [6]:
app.run_server(debug=True, mode="inline")
